## Super-resolve output

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.backends.cudnn as cudnn
from PIL import Image

from torchvision.transforms import ToTensor
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Path to data

#low_res_img_path = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-28/test/AFW_37096733_1.jpg'
low_res_img_path = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-crap-112/test/AFW_37096733_1.jpg'

original_img_path = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/test/AFW_37096733_1.jpg'

model_path = 'models/SRCNN_model_300LFW_x2_224.pth'
#model_path = 'models/SubPixelCNN_model_300LFW_x4_112.pth'
#model_path = 'models/SRGAN_Generator_model_300LFW_x4_112.pth'
#model_path = 'models/EDSR_model_300LFW_x4_112.pth'
#model_path = 'models/VDSR_model_300LFW_x4_112.pth'
#model_path = 'models/FSRCNN_model_300LFW_x4_112.pth'

In [3]:
# Input Image Setting

GPU_IN_USE = torch.cuda.is_available()
img = Image.open(low_res_img_path).convert('YCbCr')
y, cb, cr = img.split()

In [4]:
# Input image setting

device = torch.device('cuda' if GPU_IN_USE else 'cpu')
model = torch.load(model_path, map_location=lambda storage, loc: storage)
model = model.to(device)
data = (ToTensor()(y)).view(1, -1, y.size[1], y.size[0])
data = data.to(device)

if GPU_IN_USE:
    cudnn.benchmark = True

In [5]:
# Output and save image

out = model(data)
out = out.cpu()
out_img_y = out.data[0].numpy()
out_img_y *= 255.0
out_img_y = out_img_y.clip(0, 255)
out_img_y = Image.fromarray(np.uint8(out_img_y[0]), mode='L')

out_img_cb = cb.resize(out_img_y.size, Image.BICUBIC)
out_img_cr = cr.resize(out_img_y.size, Image.BICUBIC)
out_img = Image.merge('YCbCr', [out_img_y, out_img_cb, out_img_cr]).convert('RGB')

#out_img.save('result.png')

In [6]:
# Checking image quality

real_img = Image.open(original_img_path)
bicubic_img = img.resize((224,224),resample=Image.BICUBIC)

# PSNR and SSIM - Bilinear Interpolation

print('PSNR and SSIM BICUBIC: %.3f and %.3f' %(psnr(np.array(bicubic_img), np.array(real_img)), 
                                               ssim(np.array(bicubic_img), np.array(real_img), multichannel=True)))
print('PSNR and SSIM MODEL:   %.3f and %.3f' %(psnr(np.array(out_img), np.array(real_img)), 
                                               ssim(np.array(out_img), np.array(real_img), multichannel=True)))

plt.figure(figsize=(15,15))
plt.subplot(1,4,1)
plt.gca().set_title('Low Res')
plt.imshow(img)
plt.subplot(1,4,2)
plt.gca().set_title('Bicubic')
plt.imshow(bicubic_img)
plt.subplot(1,4,3)
plt.gca().set_title('Neural Net')
plt.imshow(out_img)
plt.subplot(1,4,4)
plt.gca().set_title('Original')
plt.imshow(real_img)
plt.show()

PSNR and SSIM BICUBIC: 10.850 and 0.441


ValueError: Input images must have the same dimensions.